In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [161]:
pd.set_option('display.max_columns', None)

# Top_1

1. color를 원핫인코딩 변경
2. clothSexCommon이 1인 값은 남자와 여자 모두 1로 변경
3. user테이블에 그 user가 산 옷들 평균내서 fit ~ thickness ~ season까지 저장

In [257]:
top_1 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_1.csv')

In [258]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [259]:
top_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [260]:
top_1.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [261]:
top_1.insert(4, "smallCategory_others", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_short", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_long", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_knit", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_sweatshirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_hoody", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_collar", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_shirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_halfshort", top_1['smallCategory'], True)

In [262]:
top_1['smallCategory_halfshort'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_1['smallCategory_shirt'] = top_1['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_1['smallCategory_collar'] = top_1['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_1['smallCategory_hoody'] = top_1['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_1['smallCategory_sweatshirt'] = top_1['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_1['smallCategory_knit'] = top_1['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_1['smallCategory_long'] = top_1['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_1['smallCategory_short'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_1['smallCategory_others'] = top_1['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [263]:
top_1 = top_1.drop('smallCategory', axis=1)

In [264]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [265]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_1[top_1.columns[i]].value_counts()[1]
    print(top_1[top_1.columns[i]].value_counts())
print(num)
print(top_1.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [266]:
for i in top_1['clothSexCommon']:
    if i == 1:
        top_1['clothSexMen'][i] = 1
        top_1['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [267]:
top_1 = top_1.drop('clothSexCommon', axis=1)

In [268]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [269]:
top_1['clothSexMen'][1] == 0

False

In [270]:
sub = list()
for i in range(43356):
    if top_1['clothSexMen'][i] == 0 and top_1['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    top_1['clothSexMen'][i] = 1
    top_1['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\1972896274.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\1972896274.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [271]:
print(top_1['clothSexMen'].value_counts())
print(top_1['clothSexWomen'].value_counts())
print(top_1.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [272]:
top_1.insert(14, "color_others", top_1['colorNo'], True)
top_1.insert(14, "color_pattern", top_1['colorNo'], True)
top_1.insert(14, "color_jean", top_1['colorNo'], True)
top_1.insert(14, "color_beige", top_1['colorNo'], True)
top_1.insert(14, "color_brown", top_1['colorNo'], True)
top_1.insert(14, "color_purple", top_1['colorNo'], True)
top_1.insert(14, "color_blue", top_1['colorNo'], True)
top_1.insert(14, "color_green", top_1['colorNo'], True)
top_1.insert(14, "color_yellow", top_1['colorNo'], True)
top_1.insert(14, "color_ivory", top_1['colorNo'], True)
top_1.insert(14, "color_orange", top_1['colorNo'], True)
top_1.insert(14, "color_pink", top_1['colorNo'], True)
top_1.insert(14, "color_red", top_1['colorNo'], True)
top_1.insert(14, "color_black", top_1['colorNo'], True)
top_1.insert(14, "color_grey", top_1['colorNo'], True)
top_1.insert(14, "color_white", top_1['colorNo'], True)

In [273]:
top_1['color_white'] = top_1['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_1['color_grey'] = top_1['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_1['color_black'] = top_1['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_1['color_red'] = top_1['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_1['color_pink'] = top_1['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_1['color_orange'] = top_1['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_1['color_ivory'] = top_1['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_1['color_yellow'] = top_1['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_1['color_green'] = top_1['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_1['color_blue'] = top_1['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_1['color_purple'] = top_1['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_1['color_brown'] = top_1['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_1['color_beige'] = top_1['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_1['color_jean'] = top_1['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_1['color_pattern'] = top_1['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_1['color_others'] = top_1['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [274]:
print(top_1['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [275]:
print(top_1['color_white'].value_counts())
print(top_1['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [276]:
print(top_1.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [277]:
top_1 = top_1.drop('colorNo', axis=1)

In [278]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_1[top_1.columns[i]].value_counts())
print(top_1.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [279]:
top_1.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,0,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],0,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,0,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,0,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,0,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,0,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [280]:
top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace(",", ""))
top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace("원", ""))
top_1['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [283]:
top_1.index = top_1['clothId']
top_1 = top_1.drop('clothId', True)

KeyError: 'clothId'

In [285]:
print(top_1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43356 entries, 1929966 to 1276479
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   largeCategory             43356 non-null  int64  
 1   largeCategoryName         43356 non-null  object 
 2   smallCategory_halfshort   43356 non-null  int64  
 3   smallCategory_shirt       43356 non-null  int64  
 4   smallCategory_collar      43356 non-null  int64  
 5   smallCategory_hoody       43356 non-null  int64  
 6   smallCategory_sweatshirt  43356 non-null  int64  
 7   smallCategory_knit        43356 non-null  int64  
 8   smallCategory_long        43356 non-null  int64  
 9   smallCategory_short       43356 non-null  int64  
 10  smallCategory_others      43356 non-null  int64  
 11  smallCategoryName         43356 non-null  object 
 12  color_white               43356 non-null  int64  
 13  color_grey                43356 non-null  int64  
 14

# Top_2

In [234]:
top_2 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_2.csv')

In [188]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
top_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
top_2.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
top_2.insert(4, "smallCategory_others", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_short", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_long", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_knit", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_sweatshirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_hoody", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_collar", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_shirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategory_halfshort", top_2['smallCategory'], True)

In [236]:
top_2['smallCategory_halfshort'] = top_2['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_2['smallCategory_shirt'] = top_2['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_2['smallCategory_collar'] = top_2['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_2['smallCategory_hoody'] = top_2['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_2['smallCategory_sweatshirt'] = top_2['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_2['smallCategory_knit'] = top_2['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_2['smallCategory_long'] = top_2['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_2['smallCategory_short'] = top_2['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_2['smallCategory_others'] = top_2['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
top_2 = top_2.drop('smallCategory', axis=1)

In [238]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_2[top_2.columns[i]].value_counts()[1]
    print(top_2[top_2.columns[i]].value_counts())
print(num)
print(top_2.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in top_2['clothSexCommon']:
    if i == 1:
        top_2['clothSexMen'][i] = 1
        top_2['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
top_2 = top_2.drop('clothSexCommon', axis=1)

In [242]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
top_2['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if top_2['clothSexMen'][i] == 0 and top_2['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    top_2['clothSexMen'][i] = 1
    top_2['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(top_2['clothSexMen'].value_counts())
print(top_2['clothSexWomen'].value_counts())
print(top_2.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
top_2.insert(14, "color_others", top_2['colorNo'], True)
top_2.insert(14, "color_pattern", top_2['colorNo'], True)
top_2.insert(14, "color_jean", top_2['colorNo'], True)
top_2.insert(14, "color_beige", top_2['colorNo'], True)
top_2.insert(14, "color_brown", top_2['colorNo'], True)
top_2.insert(14, "color_purple", top_2['colorNo'], True)
top_2.insert(14, "color_blue", top_2['colorNo'], True)
top_2.insert(14, "color_green", top_2['colorNo'], True)
top_2.insert(14, "color_yellow", top_2['colorNo'], True)
top_2.insert(14, "color_ivory", top_2['colorNo'], True)
top_2.insert(14, "color_orange",top_2['colorNo'], True)
top_2.insert(14, "color_pink", top_2['colorNo'], True)
top_2.insert(14, "color_red", top_2['colorNo'], True)
top_2.insert(14, "color_black", top_2['colorNo'], True)
top_2.insert(14, "color_grey", top_2['colorNo'], True)
top_2.insert(14, "color_white", top_2['colorNo'], True)

In [247]:
top_2['color_white'] = top_2['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_2['color_grey'] = top_2['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_2['color_black'] = top_2['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_2['color_red'] = top_2['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_2['color_pink'] = top_2['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_2['color_orange'] = top_2['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_2['color_ivory'] = top_2['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_2['color_yellow'] = top_2['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_2['color_green'] = top_2['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_2['color_blue'] = top_2['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_2['color_purple'] = top_2['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_2['color_brown'] = top_2['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_2['color_beige'] = top_2['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_2['color_jean'] = top_2['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_2['color_pattern'] = top_2['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_2['color_others'] = top_2['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(top_2['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(top_2['color_white'].value_counts())
print(top_2['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(top_2.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
top_2 = top_2.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_2[top_2.columns[i]].value_counts())
print(top_2.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
top_2.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace(",", ""))
top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace("원", ""))
top_2['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
top_2.index = top_2['clothId']
top_2 = top_2.drop('clothId', True)
print(top_2)

# Top_3

In [234]:
top_3 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_3.csv')

In [188]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
top_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
top_3.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
top_3.insert(4, "smallCategory_others", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_short", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_long", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_knit", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_sweatshirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_hoody", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_collar", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_shirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategory_halfshort", top_3['smallCategory'], True)

In [236]:
top_3['smallCategory_halfshort'] = top_3['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_3['smallCategory_shirt'] = top_3['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_3['smallCategory_collar'] = top_3['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_3['smallCategory_hoody'] = top_3['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_3['smallCategory_sweatshirt'] = top_3['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_3['smallCategory_knit'] = top_3['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_3['smallCategory_long'] = top_3['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_3['smallCategory_short'] = top_3['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_3['smallCategory_others'] = top_3['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
top_3 = top_3.drop('smallCategory', axis=1)

In [238]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_3[top_3.columns[i]].value_counts()[1]
    print(top_3[top_3.columns[i]].value_counts())
print(num)
print(top_3.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in top_3['clothSexCommon']:
    if i == 1:
        top_3['clothSexMen'][i] = 1
        top_3['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
top_3 = top_3.drop('clothSexCommon', axis=1)

In [242]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
top_3['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if top_3['clothSexMen'][i] == 0 and top_3['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    top_3['clothSexMen'][i] = 1
    top_3['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(top_3['clothSexMen'].value_counts())
print(top_3['clothSexWomen'].value_counts())
print(top_3.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
top_3.insert(14, "color_others", top_3['colorNo'], True)
top_3.insert(14, "color_pattern", top_3['colorNo'], True)
top_3.insert(14, "color_jean", top_3['colorNo'], True)
top_3.insert(14, "color_beige", top_3['colorNo'], True)
top_3.insert(14, "color_brown", top_3['colorNo'], True)
top_3.insert(14, "color_purple", top_3['colorNo'], True)
top_3.insert(14, "color_blue", top_3['colorNo'], True)
top_3.insert(14, "color_green", top_3['colorNo'], True)
top_3.insert(14, "color_yellow", top_3['colorNo'], True)
top_3.insert(14, "color_ivory", top_3['colorNo'], True)
top_3.insert(14, "color_orange", top_3['colorNo'], True)
top_3.insert(14, "color_pink", top_3['colorNo'], True)
top_3.insert(14, "color_red", top_3['colorNo'], True)
top_3.insert(14, "color_black", top_3['colorNo'], True)
top_3.insert(14, "color_grey", top_3['colorNo'], True)
top_3.insert(14, "color_white", top_3['colorNo'], True)

In [247]:
top_3['color_white'] = top_3['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_3['color_grey'] = top_3['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_3['color_black'] = top_3['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_3['color_red'] = top_3['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_3['color_pink'] = top_3['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_3['color_orange'] = top_3['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_3['color_ivory'] = top_3['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_3['color_yellow'] = top_3['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_3['color_green'] = top_3['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_3['color_blue'] = top_3['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_3['color_purple'] = top_3['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_3['color_brown'] = top_3['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_3['color_beige'] = top_3['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_3['color_jean'] = top_3['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_3['color_pattern'] = top_3['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_3['color_others'] = top_3['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(top_3['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(top_3['color_white'].value_counts())
print(top_3['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(top_3.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
top_3 = top_3.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_3[top_3.columns[i]].value_counts())
print(top_3.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
top_3.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace(",", ""))
top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace("원", ""))
top_3['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
top_3.index = top_3['clothId']
top_3 = top_3.drop('clothId', True)
print(top_3)

# Top_4

In [234]:
top_4 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_4.csv')

In [188]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
top_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
top_4.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
top_4.insert(4, "smallCategory_others", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_short", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_long", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_knit", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_sweatshirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_hoody", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_collar", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_shirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategory_halfshort", top_4['smallCategory'], True)

In [236]:
top_4['smallCategory_halfshort'] = top_4['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_4['smallCategory_shirt'] = top_4['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_4['smallCategory_collar'] = top_4['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_4['smallCategory_hoody'] = top_4['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_4['smallCategory_sweatshirt'] = top_4['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_4['smallCategory_knit'] = top_4['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_4['smallCategory_long'] = top_4['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_4['smallCategory_short'] = top_4['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_4['smallCategory_others'] = top_4['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
top_4 = top_4.drop('smallCategory', axis=1)

In [238]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_4[top_4.columns[i]].value_counts()[1]
    print(top_4[top_4.columns[i]].value_counts())
print(num)
print(top_4.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in top_4['clothSexCommon']:
    if i == 1:
        top_4['clothSexMen'][i] = 1
        top_4['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
top_4 = top_4.drop('clothSexCommon', axis=1)

In [242]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
top_4['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if top_4['clothSexMen'][i] == 0 and top_4['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    top_4['clothSexMen'][i] = 1
    top_4['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(top_4['clothSexMen'].value_counts())
print(top_4['clothSexWomen'].value_counts())
print(top_4.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
top_4.insert(14, "color_others", top_4['colorNo'], True)
top_4.insert(14, "color_pattern", top_4['colorNo'], True)
top_4.insert(14, "color_jean", top_4['colorNo'], True)
top_4.insert(14, "color_beige", top_4['colorNo'], True)
top_4.insert(14, "color_brown", top_4['colorNo'], True)
top_4.insert(14, "color_purple", top_4['colorNo'], True)
top_4.insert(14, "color_blue", top_4['colorNo'], True)
top_4.insert(14, "color_green", top_4['colorNo'], True)
top_4.insert(14, "color_yellow", top_4['colorNo'], True)
top_4.insert(14, "color_ivory", top_4['colorNo'], True)
top_4.insert(14, "color_orange",top_4['colorNo'], True)
top_4.insert(14, "color_pink", top_4['colorNo'], True)
top_4.insert(14, "color_red", top_4['colorNo'], True)
top_4.insert(14, "color_black", top_4['colorNo'], True)
top_4.insert(14, "color_grey", top_4['colorNo'], True)
top_4.insert(14, "color_white", top_4['colorNo'], True)

In [247]:
top_4['color_white'] = top_4['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_4['color_grey'] = top_4['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_4['color_black'] = top_4['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_4['color_red'] = top_4['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_4['color_pink'] = top_4['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_4['color_orange'] = top_4['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_4['color_ivory'] = top_4['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_4['color_yellow'] = top_4['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_4['color_green'] = top_4['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_4['color_blue'] = top_4['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_4['color_purple'] = top_4['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_4['color_brown'] = top_4['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_4['color_beige'] = top_4['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_4['color_jean'] = top_4['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_4['color_pattern'] = top_4['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_4['color_others'] = top_4['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(top_4['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(top_4['color_white'].value_counts())
print(top_4['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(top_4.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
top_4 = top_4.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_4[top_4.columns[i]].value_counts())
print(top_4.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
top_4.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace(",", ""))
top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace("원", ""))
top_4['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
top_4.index = top_4['clothId']
top_4 = top_4.drop('clothId', True)
print(top_4)

# Outer

In [234]:
outer = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_outer.csv')

In [188]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
outer.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
outer.insert(4, "smallCategory_others", outer['smallCategory'], True)
outer.insert(4, "smallCategory_short", outer['smallCategory'], True)
outer.insert(4, "smallCategory_long", outer['smallCategory'], True)
outer.insert(4, "smallCategory_knit", outer['smallCategory'], True)
outer.insert(4, "smallCategory_sweatshirt", outer['smallCategory'], True)
outer.insert(4, "smallCategory_hoody", outer['smallCategory'], True)
outer.insert(4, "smallCategory_collar", outer['smallCategory'], True)
outer.insert(4, "smallCategory_shirt", outer['smallCategory'], True)
outer.insert(4, "smallCategory_halfshort", outer['smallCategory'], True)

In [236]:
outer['smallCategory_halfshort'] = outer['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
outer['smallCategory_shirt'] = outer['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
outer['smallCategory_collar'] = outer['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
outer['smallCategory_hoody'] = outer['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
outer['smallCategory_sweatshirt'] = outer['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
outer['smallCategory_knit'] = outer['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
outer['smallCategory_long'] = outer['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
outer['smallCategory_short'] = outer['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
outer['smallCategory_others'] = outer['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
outer = outer.drop('smallCategory', axis=1)

In [238]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += outer[outer.columns[i]].value_counts()[1]
    print(outer[outer.columns[i]].value_counts())
print(num)
print(outer.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in outer['clothSexCommon']:
    if i == 1:
        outer['clothSexMen'][i] = 1
        outer['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
outer = outer.drop('clothSexCommon', axis=1)

In [242]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
outer['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if outer['clothSexMen'][i] == 0 and outer['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    outer['clothSexMen'][i] = 1
    outer['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(outer['clothSexMen'].value_counts())
print(outer['clothSexWomen'].value_counts())
print(outer.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
outer.insert(14, "color_others", outer['colorNo'], True)
outer.insert(14, "color_pattern", outer['colorNo'], True)
outer.insert(14, "color_jean", outer['colorNo'], True)
outer.insert(14, "color_beige", outer['colorNo'], True)
outer.insert(14, "color_brown", outer['colorNo'], True)
outer.insert(14, "color_purple", outer['colorNo'], True)
outer.insert(14, "color_blue", outer['colorNo'], True)
outer.insert(14, "color_green", outer['colorNo'], True)
outer.insert(14, "color_yellow", outer['colorNo'], True)
outer.insert(14, "color_ivory", outer['colorNo'], True)
outer.insert(14, "color_orange", outer['colorNo'], True)
outer.insert(14, "color_pink", outer['colorNo'], True)
outer.insert(14, "color_red", outer['colorNo'], True)
outer.insert(14, "color_black", outer['colorNo'], True)
outer.insert(14, "color_grey", outer['colorNo'], True)
outer.insert(14, "color_white", outer['colorNo'], True)

In [247]:
outer['color_white'] = outer['color_white'].apply(lambda x: 1 if x == 1 else 0)
outer['color_grey'] = outer['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
outer['color_black'] = outer['color_black'].apply(lambda x: 1 if x == 2 else 0)
outer['color_red'] = outer['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
outer['color_pink'] = outer['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
outer['color_orange'] = outer['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
outer['color_ivory'] = outer['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
outer['color_yellow'] = outer['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
outer['color_green'] = outer['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
outer['color_blue'] = outer['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
outer['color_purple'] = outer['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
outer['color_brown'] = outer['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
outer['color_beige'] = outer['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
outer['color_jean'] = outer['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
outer['color_pattern'] = outer['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
outer['color_others'] = outer['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(outer['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(outer['color_white'].value_counts())
print(outer['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(outer.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
outer = outer.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(outer[outer.columns[i]].value_counts())
print(outer.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
outer.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace(",", ""))
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace("원", ""))
outer['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
outer.index = outer['clothId']
outer = outer.drop('clothId', True)
print(outer)

# Pants

In [234]:
pants = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_pants.csv')

In [188]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
pants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
pants.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
pants.insert(4, "smallCategory_others", pants['smallCategory'], True)
pants.insert(4, "smallCategory_short", pants['smallCategory'], True)
pants.insert(4, "smallCategory_long", pants['smallCategory'], True)
pants.insert(4, "smallCategory_knit", pants['smallCategory'], True)
pants.insert(4, "smallCategory_sweatshirt", pants['smallCategory'], True)
pants.insert(4, "smallCategory_hoody", pants['smallCategory'], True)
pants.insert(4, "smallCategory_collar", pants['smallCategory'], True)
pants.insert(4, "smallCategory_shirt", pants['smallCategory'], True)
pants.insert(4, "smallCategory_halfshort", pants['smallCategory'], True)

In [236]:
pants['smallCategory_halfshort'] = pants['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
pants['smallCategory_shirt'] = pants['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
pants['smallCategory_collar'] = pants['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
pants['smallCategory_hoody'] = pants['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
pants['smallCategory_sweatshirt'] = pants['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
pants['smallCategory_knit'] = pants['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
pants['smallCategory_long'] = pants['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
pants['smallCategory_short'] = pants['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
pants['smallCategory_others'] = pants['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
pants = pants.drop('smallCategory', axis=1)

In [238]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += pants[pants.columns[i]].value_counts()[1]
    print(pants[pants.columns[i]].value_counts())
print(num)
print(pants.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in pants['clothSexCommon']:
    if i == 1:
        pants['clothSexMen'][i] = 1
        pants['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
pants = pants.drop('clothSexCommon', axis=1)

In [242]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
pants['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if pants['clothSexMen'][i] == 0 and pants['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    pants['clothSexMen'][i] = 1
    pants['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(pants['clothSexMen'].value_counts())
print(pants['clothSexWomen'].value_counts())
print(pants.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
pants.insert(14, "color_others", pants['colorNo'], True)
pants.insert(14, "color_pattern", pants['colorNo'], True)
pants.insert(14, "color_jean", pants['colorNo'], True)
pants.insert(14, "color_beige", pants['colorNo'], True)
pants.insert(14, "color_brown", pants['colorNo'], True)
pants.insert(14, "color_purple", pants['colorNo'], True)
pants.insert(14, "color_blue", pants['colorNo'], True)
pants.insert(14, "color_green", pants['colorNo'], True)
pants.insert(14, "color_yellow", pants['colorNo'], True)
pants.insert(14, "color_ivory", pants['colorNo'], True)
pants.insert(14, "color_orange", pants['colorNo'], True)
pants.insert(14, "color_pink", pants['colorNo'], True)
pants.insert(14, "color_red", pants['colorNo'], True)
pants.insert(14, "color_black", pants['colorNo'], True)
pants.insert(14, "color_grey", pants['colorNo'], True)
pants.insert(14, "color_white", pants['colorNo'], True)

In [247]:
pants['color_white'] = pants['color_white'].apply(lambda x: 1 if x == 1 else 0)
pants['color_grey'] = pants['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
pants['color_black'] = pants['color_black'].apply(lambda x: 1 if x == 2 else 0)
pants['color_red'] = pants['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
pants['color_pink'] = pants['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
pants['color_orange'] = pants['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
pants['color_ivory'] = pants['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
pants['color_yellow'] = pants['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
pants['color_green'] = pants['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
pants['color_blue'] = pants['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
pants['color_purple'] = pants['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
pants['color_brown'] = pants['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
pants['color_beige'] = pants['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
pants['color_jean'] = pants['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
pants['color_pattern'] = pants['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
pants['color_others'] = pants['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(pants['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(pants['color_white'].value_counts())
print(pants['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(pants.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
pants = pants.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(pants[pants.columns[i]].value_counts())
print(pants.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
pants.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace(",", ""))
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace("원", ""))
pants['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
pants.index = pants['clothId']
pants = pants.drop('clothId', True)
print(pants)

# Onepiece

In [234]:
onepiece = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_onepiece.csv')

In [188]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
onepiece.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
onepiece.insert(4, "smallCategory_others", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_short", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_long", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_knit", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_sweatshirt", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_hoody", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_collar", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_shirt", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategory_halfshort", onepiece['smallCategory'], True)

In [236]:
onepiece['smallCategory_halfshort'] = onepiece['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
onepiece['smallCategory_shirt'] = onepiece['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
onepiece['smallCategory_collar'] = onepiece['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
onepiece['smallCategory_hoody'] = onepiece['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
onepiece['smallCategory_sweatshirt'] = onepiece['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
onepiece['smallCategory_knit'] = onepiece['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
onepiece['smallCategory_long'] = onepiece['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
onepiece['smallCategory_short'] = onepiece['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
onepiece['smallCategory_others'] = onepiece['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
onepiece = onepiece.drop('smallCategory', axis=1)

In [238]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += onepiece[onepiece.columns[i]].value_counts()[1]
    print(onepiece[onepiece.columns[i]].value_counts())
print(num)
print(onepiece.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in onepiece['clothSexCommon']:
    if i == 1:
        onepiece['clothSexMen'][i] = 1
        onepiece['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
onepiece = onepiece.drop('clothSexCommon', axis=1)

In [242]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
onepiece['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if onepiece['clothSexMen'][i] == 0 and onepiece['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    onepiece['clothSexMen'][i] = 1
    onepiece['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(onepiece['clothSexMen'].value_counts())
print(onepiece['clothSexWomen'].value_counts())
print(onepiece.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
onepiece.insert(14, "color_others", onepiece['colorNo'], True)
onepiece.insert(14, "color_pattern", onepiece['colorNo'], True)
onepiece.insert(14, "color_jean", onepiece['colorNo'], True)
onepiece.insert(14, "color_beige", onepiece['colorNo'], True)
onepiece.insert(14, "color_brown", onepiece['colorNo'], True)
onepiece.insert(14, "color_purple", onepiece['colorNo'], True)
onepiece.insert(14, "color_blue", onepiece['colorNo'], True)
onepiece.insert(14, "color_green", onepiece['colorNo'], True)
onepiece.insert(14, "color_yellow", onepiece['colorNo'], True)
onepiece.insert(14, "color_ivory", onepiece['colorNo'], True)
onepiece.insert(14, "color_orange", onepiece['colorNo'], True)
onepiece.insert(14, "color_pink", onepiece['colorNo'], True)
onepiece.insert(14, "color_red", onepiece['colorNo'], True)
onepiece.insert(14, "color_black", onepiece['colorNo'], True)
onepiece.insert(14, "color_grey", onepiece['colorNo'], True)
onepiece.insert(14, "color_white", onepiece['colorNo'], True)

In [247]:
onepiece['color_white'] = onepiece['color_white'].apply(lambda x: 1 if x == 1 else 0)
onepiece['color_grey'] = onepiece['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
onepiece['color_black'] = onepiece['color_black'].apply(lambda x: 1 if x == 2 else 0)
onepiece['color_red'] = onepiece['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
onepiece['color_pink'] = onepiece['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
onepiece['color_orange'] = onepiece['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
onepiece['color_ivory'] = onepiece['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
onepiece['color_yellow'] = onepiece['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
onepiece['color_green'] = onepiece['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
onepiece['color_blue'] = onepiece['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
onepiece['color_purple'] = onepiece['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
onepiece['color_brown'] = onepiece['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
onepiece['color_beige'] = onepiece['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
onepiece['color_jean'] = onepiece['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
onepiece['color_pattern'] = onepiece['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
onepiece['color_others'] = onepiece['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(onepiece['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(onepiece['color_white'].value_counts())
print(onepiece['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(onepiece.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
onepiece = onepiece.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(onepiece[onepiece.columns[i]].value_counts())
print(onepiece.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
onepiece.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace(",", ""))
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace("원", ""))
onepiece['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
onepiece.index = onepiece['clothId']
onepiece = onepiece.drop('clothId', True)
print(onepiece)

# Skirt

In [234]:
skirt = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_skirt.csv')

In [188]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
skirt.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [235]:
skirt.insert(4, "smallCategory_others", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_short", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_long", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_knit", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_sweatshirt", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_hoody", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_collar", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_shirt", skirt['smallCategory'], True)
skirt.insert(4, "smallCategory_halfshort", skirt['smallCategory'], True)

In [236]:
skirt['smallCategory_halfshort'] = skirt['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
skirt['smallCategory_shirt'] = skirt['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
skirt['smallCategory_collar'] = skirt['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
skirt['smallCategory_hoody'] = skirt['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
skirt['smallCategory_sweatshirt'] = skirt['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
skirt['smallCategory_knit'] = skirt['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
skirt['smallCategory_long'] = skirt['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
skirt['smallCategory_short'] = skirt['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
skirt['smallCategory_others'] = skirt['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [237]:
skirt = skirt.drop('smallCategory', axis=1)

In [238]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [239]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += skirt[skirt.columns[i]].value_counts()[1]
    print(skirt[skirt.columns[i]].value_counts())
print(num)
print(skirt.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [240]:
for i in skirt['clothSexCommon']:
    if i == 1:
        skirt['clothSexMen'][i] = 1
        skirt['clothSexWomen'][i] = 1

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\3641113567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [241]:
skirt = skirt.drop('clothSexCommon', axis=1)

In [242]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [243]:
skirt['clothSexMen'][1] == 0

False

In [244]:
sub = list()
for i in range(43356):
    if skirt['clothSexMen'][i] == 0 and skirt['clothSexWomen'][i] == 0:
        sub.append(i)
    else:
        pass
for i in sub:
    skirt['clothSexMen'][i] = 1
    skirt['clothSexWomen'][i] = 1

[27, 28, 169, 181, 182, 193, 268, 286, 306, 349, 369, 384, 411, 658, 761, 762, 843, 851, 915, 928, 934, 941, 953, 957, 1005, 1026, 1046, 1068, 1100, 1103, 1139, 1207, 1224, 1225, 1265, 1267, 1272, 1275, 1297, 1313, 1327, 1337, 1346, 1356, 1358, 1360, 1362, 1367, 1390, 1407, 1414, 1416, 1426, 1643, 1692, 1695, 1709, 1725, 1849, 1865, 2006, 2075, 2107, 2108, 2258, 2283, 2321, 2326, 2328, 2331, 2332, 2397, 2493, 2607, 2612, 2613, 2615, 2617, 2618, 2620, 2689, 2698, 2704, 2731, 2758, 2767, 2774, 2794, 2799, 2812, 2816, 2838, 2849, 2853, 2858, 2865, 2873, 2874, 2877, 2919, 3062, 3110, 3119, 3358, 3362, 3363, 3414, 3433, 3513, 3535, 3536, 3537, 3572, 3600, 3609, 3612, 3635, 3645, 3707, 3762, 3776, 3834, 3868, 3913, 3951, 3963, 3993, 4033, 4114, 4167, 4206, 4210, 4236, 4238, 4250, 4258, 4259, 4262, 4267, 4269, 4282, 4309, 4384, 4401, 4569, 4593, 4643, 4749, 4751, 4755, 4781, 4802, 4834, 4842, 4844, 4882, 4883, 4922, 4934, 4972, 4976, 4981, 4982, 4985, 4996, 4998, 5017, 5038, 5111, 5114, 5118,

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexMen'][i] = 1
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12604\855514687.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1['clothSexWomen'][i] = 1


In [245]:
print(skirt['clothSexMen'].value_counts())
print(skirt['clothSexWomen'].value_counts())
print(skirt.shape)

1    32247
0    11109
Name: clothSexMen, dtype: int64
0    30600
1    12756
Name: clothSexWomen, dtype: int64
(43356, 33)


### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [246]:
skirt.insert(14, "color_others", skirt['colorNo'], True)
skirt.insert(14, "color_pattern", skirt['colorNo'], True)
skirt.insert(14, "color_jean", skirt['colorNo'], True)
skirt.insert(14, "color_beige", skirt['colorNo'], True)
skirt.insert(14, "color_brown", skirt['colorNo'], True)
skirt.insert(14, "color_purple", skirt['colorNo'], True)
skirt.insert(14, "color_blue", skirt['colorNo'], True)
skirt.insert(14, "color_green", skirt['colorNo'], True)
skirt.insert(14, "color_yellow", skirt['colorNo'], True)
skirt.insert(14, "color_ivory", skirt['colorNo'], True)
skirt.insert(14, "color_orange", skirt['colorNo'], True)
skirt.insert(14, "color_pink", skirt['colorNo'], True)
skirt.insert(14, "color_red", skirt['colorNo'], True)
skirt.insert(14, "color_black", skirt['colorNo'], True)
skirt.insert(14, "color_grey", skirt['colorNo'], True)
skirt.insert(14, "color_white", skirt['colorNo'], True)

In [247]:
skirt['color_white'] = skirt['color_white'].apply(lambda x: 1 if x == 1 else 0)
skirt['color_grey'] = skirt['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
skirt['color_black'] = skirt['color_black'].apply(lambda x: 1 if x == 2 else 0)
skirt['color_red'] = skirt['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
skirt['color_pink'] = skirt['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
skirt['color_orange'] = skirt['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
skirt['color_ivory'] = skirt['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
skirt['color_yellow'] = skirt['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
skirt['color_green'] = skirt['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
skirt['color_blue'] = skirt['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
skirt['color_purple'] = skirt['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
skirt['color_brown'] = skirt['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
skirt['color_beige'] = skirt['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
skirt['color_jean'] = skirt['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
skirt['color_pattern'] = skirt['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
skirt['color_others'] = skirt['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [248]:
print(skirt['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [249]:
print(skirt['color_white'].value_counts())
print(skirt['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [250]:
print(skirt.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [251]:
skirt = skirt.drop('colorNo', axis=1)

In [252]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(skirt[skirt.columns[i]].value_counts())
print(skirt.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
(43356, 48)


In [165]:
skirt.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0


### Price 전처리

In [254]:
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace(",", ""))
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace("원", ""))
skirt['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43351    63200
43352    69000
43353    32200
43354    79000
43355    26900
Name: clothPrice, Length: 43356, dtype: object

In [ ]:
skirt.index = skirt['clothId']
skirt = skirt.drop('clothId', True)
print(skirt)